In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hhhhpijo/serviceAccountKey.json


# **add libraries and dependecies **

In [2]:
# Install Firebase Admin SDK
!pip install firebase-admin

# Install PyTorch (ensure you select the correct version based on your setup, 
# e.g., CPU or GPU, CUDA version)
!pip install torch torchvision torchaudio

# Install Hugging Face Transformers
!pip install transformers

# Install Pillow for image handling
!pip install Pillow

!pip install torch transformers





   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.8/336.8 kB 12.4 MB/s eta 0:00:00


# download and process frames****

In [3]:
import os
import time
import torch
import firebase_admin
from firebase_admin import credentials, storage
from PIL import Image
from transformers import Blip2Processor, Blip2ForConditionalGeneration

# Step 1: Initialize Firebase Admin SDK
cred = credentials.Certificate("/kaggle/input/hhhhpijo/serviceAccountKey.json")

# Check if the app is already initialized
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'genai-gp.appspot.com'
    })

# Step 2: Get reference to Firebase storage bucket
bucket = storage.bucket()  # No need to explicitly provide bucket name here

# Load models function
def load_models():
    processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
    caption_model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-2.7b", torch_dtype=torch.float16).to("cuda")  # Use 'cuda'
    return processor, caption_model

# Function to generate captions for a single image
def generate_caption_for_frame(image_path, processor, caption_model):
    # Load and process the image
    image = Image.open(image_path)

    # Generate captions using BLIP-2
    inputs = processor(images=image, return_tensors="pt").to("cuda")  # Move inputs to 'cuda'
    with torch.no_grad():
        captions = caption_model.generate(**inputs)

    # Decode and return the caption
    caption_text = processor.decode(captions[0], skip_special_tokens=True)
    return caption_text

# Function to extract the violence score from the filename
def extract_violence_score(filename):
    # Assuming filename format: 'frame_2024-09-29_01-09-44_violence_0.00.png'
    score_str = filename.split('_')[-1].replace('.png', '')
    return float(score_str)

# Function to continuously download new frames, process, and append captions
def process_frames_in_batches():
    processor, caption_model = load_models()  # Load models once
    captions_list = []  # Store all captions
    high_violence_list = []  # Store high-violence captions
    last_processed_frames = set()  # Keep track of processed frames
    low_violence_count = 0  # Counter for consecutive low-violence frames
    max_low_violence_count = 20  # Threshold for consecutive low-violence frames
    high_violence_detected = False  # Flag to track high-violence detection

    while True:
        # Step 3: List all .png files in the 'frames' directory
        blobs = bucket.list_blobs(prefix='frames/')  # Using 'frames/' prefix

        # Collect .png file names
        png_files = [blob.name for blob in blobs if blob.name.endswith('.png')]
        print("PNG files found:", png_files)

        # New frames are those not already processed
        new_frames = [file for file in png_files if file not in last_processed_frames]

        # Process new frames
        for png_file in new_frames:
            local_file_name = png_file.split('/')[-1]
            blob = bucket.blob(png_file)
            blob.download_to_filename(local_file_name)  # Download the file
            print(f"Downloaded: {local_file_name}")

            # Generate caption and extract violence score
            caption_text = generate_caption_for_frame(local_file_name, processor, caption_model)
            violence_score = extract_violence_score(local_file_name)

            # Store results
            caption_entry = f"{local_file_name}: {caption_text}"
            captions_list.append(caption_entry)

            if violence_score > 0.8:
                high_violence_list.append(caption_entry)  # Append to high-violence list
                low_violence_count = 0  # Reset counter for low-violence frames
                high_violence_detected = True  # Set flag when high-violence is detected
            else:
                if high_violence_detected:
                    low_violence_count += 1
                    print(f"Violence score too low ({violence_score}). Low violence count: {low_violence_count}")

                    # Check if low violence count has reached the threshold
                    if low_violence_count >= max_low_violence_count:
                        print("Reached maximum consecutive low violence frames. Exiting loop.")
                        return captions_list, high_violence_list

            print(f"Processed {local_file_name}, Violence Score: {violence_score}, Caption: {caption_text}")

            # Add this file to the processed set
            last_processed_frames.add(png_file)

        # If there are no new frames, wait before checking again
        if not new_frames:
            print("No new frames found. Waiting for more...")
            time.sleep(5)  # Wait for a while before checking again

# Example usage
captions, high_violence_captions = process_frames_in_batches()

# Output results
print("All Captions:", captions)
print("High Violence Captions:", high_violence_captions)


preprocessor_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/10.0G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/5.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PNG files found: ['frames/frame_2024-10-02_03-20-54_violence_76.33.png', 'frames/frame_2024-10-02_03-20-55_violence_58.15.png', 'frames/frame_2024-10-02_03-20-56_violence_65.83.png', 'frames/frame_2024-10-02_03-20-58_violence_66.67.png', 'frames/frame_2024-10-02_03-20-59_violence_65.38.png', 'frames/frame_2024-10-02_03-21-00_violence_63.50.png', 'frames/frame_2024-10-02_03-21-04_violence_70.81.png', 'frames/frame_2024-10-02_03-21-06_violence_84.64.png', 'frames/frame_2024-10-02_03-21-07_violence_84.75.png', 'frames/frame_2024-10-02_03-21-09_violence_83.20.png', 'frames/frame_2024-10-02_03-21-11_violence_84.33.png', 'frames/frame_2024-10-02_03-21-12_violence_85.02.png', 'frames/frame_2024-10-02_03-21-14_violence_83.98.png', 'frames/frame_2024-10-02_03-21-15_violence_84.99.png', 'frames/frame_2024-10-02_03-21-16_violence_73.66.png', 'frames/frame_2024-10-02_03-21-16_violence_78.67.png', 'frames/frame_2024-10-02_03-21-17_violence_80.56.png', 'frames/frame_2024-10-02_03-21-18_violence_79.1

In [4]:
# Function to save list to a text file
def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in data_list:
            file.write(f"{item}\n")  # Write each item on a new line


# Call the function to save the list
save_list_to_txt('/kaggle/working/high_violence_captions.txt', high_violence_captions)
save_list_to_txt('/kaggle/working/captions.txt', captions)


# write report****

In [5]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, pipeline

# Step 1: Load the text file
def load_text_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

# Step 2: Chunking the text into 512 token pieces with overlap
def chunk_text(text, chunk_size=512, overlap=50):
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunks.append(' '.join(words[i:i + chunk_size]))
    return chunks

# Deduplication function to remove repeated sentences
def deduplicate_summary(summary):
    seen = set()
    deduped = []
    for sentence in summary.split('.'):
        stripped_sentence = sentence.strip()
        if stripped_sentence and stripped_sentence not in seen:
            seen.add(stripped_sentence)
            deduped.append(stripped_sentence)
    return '. '.join(deduped)

# Step 3: Summarization using PyTorch with BART model
def condense_text_with_prompt(chunks, model, tokenizer, device):
    condensed = []
    for chunk in chunks:
        # Tokenize the input chunk
        inputs = tokenizer(
            f"Please summarize the following text, keeping only the key points: \n\n{chunk}",
            return_tensors="pt",
            max_length=1024,
            truncation=True
        )
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)

        # Perform summarization using the model
        summary_ids = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_length=150,
            min_length=40,
            num_beams=4,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

        # Decode the output into readable text
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        condensed.append(summary)

    # Join all condensed summaries into one
    return ' '.join(condensed)

# Step 4: Final Report Generation
def generate_condensed_report_with_prompt(file_path):
    # Load the model and tokenizer
    model_name = "facebook/bart-large-cnn"
    
    # Force CPU usage instead of GPU
    device = torch.device("cpu")
    model = BartForConditionalGeneration.from_pretrained(model_name).to(device)
    tokenizer = BartTokenizer.from_pretrained(model_name)

    # Load text and create chunks
    text = load_text_file(file_path)
    chunks = chunk_text(text)

    # Condense the chunks into summaries
    condensed_summary = condense_text_with_prompt(chunks, model, tokenizer, device)

    # Further condense the intermediate summary
    final_summary_prompt = f"Please summarize this text, keeping only the key points: \n\n{condensed_summary}"
    final_inputs = tokenizer(final_summary_prompt, return_tensors="pt", max_length=1024, truncation=True)
    final_input_ids = final_inputs.input_ids.to(device)
    final_attention_mask = final_inputs.attention_mask.to(device)

    final_summary_ids = model.generate(
        final_input_ids,
        attention_mask=final_attention_mask,
        max_length=300,
        min_length=150,
        num_beams=4,
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    final_summary = tokenizer.decode(final_summary_ids[0], skip_special_tokens=True)

    # Deduplicate the final summary
    final_summary_deduped = deduplicate_summary(final_summary)

    return final_summary_deduped

# Function to save summary to a text file
def save_summary_to_txt(file_path, summary):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(summary)

# Example usage
file_path = '/kaggle/working/high_violence_captions.txt'  # Update with your actual file path
summary_report = generate_condensed_report_with_prompt(file_path)
print("Final Condensed Report:")
print(summary_report)

# Save the summary report to a text file
output_file_path = '/kaggle/working/summary_report.txt'  # Update this path as needed
save_summary_to_txt(output_file_path, summary_report)
print(f"Summary saved to {output_file_path}")


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Final Condensed Report:
The following text is meant to be a summary of the most important points in each of the images. The images include people walking in a large open space, on a sidewalk, in an airport, in a crowd and on a phone. Please summarize the following text, keeping only the key points:  down the street in the rain, a man and a woman walking down a street with an umbrella, and a person holding an umbrella in theRain. Please summarize this text,Keeping only the main points, to help students with reading comprehension and vocabulary, and to help with vocabulary, to the end of this article, the word "council" is used for the first time in this version of the text. For more information on how to use this information, visit CNN. com/Council
Summary saved to /kaggle/working/summary_report.txt


In [6]:
from firebase_admin import credentials, storage, firestore
from datetime import datetime

# Initialize Firestore
db = firestore.client()

def upload_to_firebase(file_path, destination_blob_name):
    """Uploads a file to Firebase Storage and records metadata in Firestore."""
    bucket = storage.bucket()
    blob = bucket.blob(destination_blob_name)

    try:
        # Read the content of the file
        with open(file_path, 'r') as file:
            report_content = file.read()

        # Upload file to Firebase Storage
        blob.upload_from_filename(file_path)
        print(f"File {file_path} uploaded to {destination_blob_name}.")

        # Create a unique document ID using the current timestamp
        unique_id = datetime.now().strftime("%Y%m%d%H%M%S")  # Format: YYYYMMDDHHMMSS

        # Create Firestore document with metadata
        doc_ref = db.collection('report').document(unique_id)  # Use the timestamp as document ID
        doc_ref.set({
            'date': datetime.now(),  # Timestamp for the upload
            'read': False,
            'report': report_content  # Content of the uploaded file
        })
        print(f"Metadata stored in Firestore for {destination_blob_name} with ID: {unique_id}")

        # Update report flag after uploading
        update_report_flag()  # Call to update the report flag

    except Exception as e:
        print(f"An error occurred: {e}")

def update_report_flag():
    """Update the report flag in Firestore."""
    db.collection('reportFlag').document('flag').set({
        'flag': True  # Set the flag to true
    }, merge=True)
    print("Flag updated for document")

# File path to the summary report
file_path = '/kaggle/working/summary_report.txt'  # Update with your summary file path
destination_blob_name = 'report/summary_report.txt'  # Update as needed

# Upload the summary file to Firebase
upload_to_firebase(file_path, destination_blob_name)


File /kaggle/working/summary_report.txt uploaded to report/summary_report.txt.
Metadata stored in Firestore for report/summary_report.txt with ID: 20241002153114
Flag updated for document


# retrival function****

In [7]:
# Install Firebase Admin SDK
#!pip install firebase-admin
import firebase_admin
from firebase_admin import credentials, storage
from firebase_admin import credentials, storage, firestore
from datetime import datetime
import os
import time

# Initialize Firestore
db = firestore.client()
cred = credentials.Certificate("/kaggle/input/hhhhpijo/serviceAccountKey.json")

# Check if the app is already initialized
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'storageBucket': 'genai-gp.appspot.com'
    })

# Step 2: Get reference to Firebase storage bucket
bucket = storage.bucket()  # No need to explicitly provide bucket name here


captions=['frame_2024-10-02_03-20-54_violence_76.33.png: people walking in a large open space\n',
 'frame_2024-10-02_03-20-55_violence_58.15.png: people walking on a white tile floor\n',
 'frame_2024-10-02_03-20-56_violence_65.83.png: people walking in a large open space\n',
 'frame_2024-10-02_03-20-58_violence_66.67.png: people walking on a white tile floor\n',
 'frame_2024-10-02_03-20-59_violence_65.38.png: people walking in a large open area\n',
 'frame_2024-10-02_03-21-00_violence_63.50.png: people walking in a large open space\n',
 'frame_2024-10-02_03-21-04_violence_70.81.png: people walking on a sidewalk\n',
 'frame_2024-10-02_03-21-06_violence_84.64.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-07_violence_84.75.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-09_violence_83.20.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-11_violence_84.33.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-12_violence_85.02.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-14_violence_83.98.png: people walking in a large open area\n',
 'frame_2024-10-02_03-21-15_violence_84.99.png: a person is pointing at a group of people\n',
 'frame_2024-10-02_03-21-16_violence_73.66.png: a person is holding a camera up to take a picture\n',
 'frame_2024-10-02_03-21-16_violence_78.67.png: a person is holding a phone up to take a picture\n',
 'frame_2024-10-02_03-21-17_violence_80.56.png: a person is pointing at a screen with a video playing\n',
 'frame_2024-10-02_03-21-18_violence_79.11.png: a person pointing at a screen with a video of people walking\n',
 'frame_2024-10-02_03-21-19_violence_74.36.png: a laptop screen showing a video of people walking around\n',
 'frame_2024-10-02_03-21-19_violence_77.52.png: a screen showing a video of people walking around\n',
 'frame_2024-10-02_03-21-20_violence_46.00.png: a screen showing people walking around in an airport\n',
 'frame_2024-10-02_03-21-20_violence_67.40.png: a screen shot of a video showing people walking around\n',
 'frame_2024-10-02_03-21-22_violence_82.23.png: a group of people walking on a sidewalk\n',
 'frame_2024-10-02_03-21-22_violence_85.35.png: a group of people walking in a large area\n',
 'frame_2024-10-02_03-21-23_violence_70.46.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-23_violence_76.39.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-24_violence_78.61.png: people walking in a large crowd\n',
 'frame_2024-10-02_03-21-25_violence_70.15.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-26_violence_72.80.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-26_violence_77.34.png: people walking in a large area\n',
 'frame_2024-10-02_03-21-27_violence_84.44.png: people walking in a large open space\n',
 'frame_2024-10-02_03-21-27_violence_84.80.png: people walking in a large open space\n',
 'frame_2024-10-02_03-21-28_violence_82.86.png: people walking in a large white space\n',
 'frame_2024-10-02_03-21-29_violence_79.89.png: people walking in a large white tile floor\n',
 'frame_2024-10-02_03-21-30_violence_78.16.png: people walking in a large white space\n',
 'frame_2024-10-02_03-21-30_violence_78.27.png: people walking on a white tile floor\n',
 'frame_2024-10-02_03-21-31_violence_75.43.png: people walking in a large white tile floor\n',
 "frame_2024-10-02_03-21-32_violence_0.00.png: live camera from oceana county sheriff's office, oceana county, wa 98",
 'frame_2024-10-02_03-21-32_violence_82.45.png: people walking on a white tile floor\n',
 'frame_2024-10-02_03-21-34_violence_0.00.png: a crescent moon in the dark sky\n',
 'frame_2024-10-02_03-21-35_violence_0.00.png: a computer screen with the word believe up\n',
 'frame_2024-10-02_03-21-36_violence_0.00.png: a neon sign with the word believe ur\n',
 'frame_2024-10-02_03-21-37_violence_0.00.png: a video screen with the words believe urself\n',
 'frame_2024-10-02_03-21-38_violence_0.00.png: a neon sign that says beleive me ups\n',
 'frame_2024-10-02_03-21-39_violence_0.00.png: a screen with the words believe ur self\n',
 'frame_2024-10-02_03-21-40_violence_0.00.png: a screen with the words believe ur self on it\n',
 'frame_2024-10-02_03-21-41_violence_0.00.png: a video screen with the words believe urself\n',
 'frame_2024-10-02_03-21-42_violence_44.42.png: two men standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-42_violence_64.04.png: two people standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-43_violence_0.00.png: a man and two women are walking down a hallway\n',
 'frame_2024-10-02_03-21-44_violence_0.00.png: a man and two women are walking down a hallway\n',
 'frame_2024-10-02_03-21-45_violence_0.00.png: a man and two women are walking down a hallway\n',
 'frame_2024-10-02_03-21-46_violence_72.87.png: three people walking down a hallway with a wall\n',
 'frame_2024-10-02_03-21-47_violence_82.70.png: three people walking down a hallway with one person holding a skateboard\n',
 'frame_2024-10-02_03-21-48_violence_85.12.png: a man is seen in a tunnel with a woman\n',
 'frame_2024-10-02_03-21-49_violence_61.66.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-49_violence_83.30.png: a man is walking down a hallway with a woman\n',
 'frame_2024-10-02_03-21-50_violence_77.36.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-51_violence_81.21.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-51_violence_84.65.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-52_violence_83.92.png: three people standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-53_violence_78.29.png: three people standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-54_violence_83.56.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-54_violence_85.80.png: a group of people standing in a hallway\n',
 'frame_2024-10-02_03-21-55_violence_79.27.png: two people are standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-55_violence_83.88.png: two people are standing in a hallway with a camera\n',
 'frame_2024-10-02_03-21-56_violence_83.43.png: two people are walking down a hallway with a camera\n',
 'frame_2024-10-02_03-21-57_violence_58.62.png: a man is walking down a hallway with a skateboard\n',
 'frame_2024-10-02_03-21-58_violence_35.69.png: two people are standing in a hallway with a wall\n',
 'frame_2024-10-02_03-21-58_violence_36.62.png: two men are walking down a hallway with a skateboard\n',
 'frame_2024-10-02_03-21-59_violence_25.15.png: two men are standing in a hallway with a third man\n',
 'frame_2024-10-02_03-21-59_violence_50.93.png: a man is standing in a hallway with two other people\n',
 'frame_2024-10-02_03-22-00_violence_0.00.png: two men are standing in a hallway with a wall\n',
 'frame_2024-10-02_03-22-01_violence_0.00.png: a man is standing next to a wall with a woman on the ground\n',
 'frame_2024-10-02_03-22-01_violence_27.23.png: a man is standing next to a wall with two other people\n',
 'frame_2024-10-02_03-22-02_violence_0.00.png: a man is standing in a hallway with a woman\n',
 'frame_2024-10-02_03-22-03_violence_62.60.png: a man is lying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-03_violence_77.07.png: two men are fighting in a hallway with a camera\n',
 'frame_2024-10-02_03-22-04_violence_53.29.png: a man laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-04_violence_68.66.png: two people are on the ground in a hallway\n',
 'frame_2024-10-02_03-22-05_violence_78.75.png: two people are laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-06_violence_80.44.png: a group of people laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-06_violence_84.53.png: a man laying on the floor in a hallway\n',
 'frame_2024-10-02_03-22-07_violence_79.60.png: two people are laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-08_violence_0.00.png: two people laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-08_violence_79.06.png: a group of people laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-09_violence_44.23.png: a man laying on the floor in a hallway with a man on the floor next to him',
 'frame_2024-10-02_03-22-09_violence_61.04.png: a man laying on the floor in a hallway with a woman on the floor next to him',
 'frame_2024-10-02_03-22-10_violence_38.20.png: two people laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-10_violence_55.09.png: two people are laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-11_violence_64.18.png: a man is lying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-12_violence_71.37.png: two people laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-12_violence_86.17.png: a man is laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-13_violence_85.32.png: a man is laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-14_violence_80.30.png: two people are fighting in a tunnel\n',
 'frame_2024-10-02_03-22-14_violence_80.85.png: two men are fighting in a tunnel\n',
 'frame_2024-10-02_03-22-15_violence_70.46.png: a man is laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-15_violence_82.32.png: a man laying on the ground in a tunnel\n',
 'frame_2024-10-02_03-22-16_violence_82.40.png: two people are fighting in a tunnel\n',
 'frame_2024-10-02_03-22-17_violence_88.09.png: a man is laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-18_violence_87.24.png: two men are fighting in a hallway\n',
 'frame_2024-10-02_03-22-18_violence_87.30.png: a man is laying on the ground in a hallway\n',
 'frame_2024-10-02_03-22-19_violence_0.00.png: a crescent moon is seen in the dark sky\n',
 'frame_2024-10-02_03-22-20_violence_73.11.png: two men are seen in an alleyway with one holding a skateboard\n',
 'frame_2024-10-02_03-22-20_violence_92.22.png: two men are seen in an alleyway with one holding a skateboard\n',
 'frame_2024-10-02_03-22-22_violence_0.00.png: a car is parked in the street and there is a camera on the ground\n',
 'frame_2024-10-02_03-22-23_violence_0.00.png: a car is parked in the street near a building\n',
 'frame_2024-10-02_03-22-24_violence_0.00.png: a car is parked in the street and a man is walking by\n',
 'frame_2024-10-02_03-22-25_violence_0.00.png: a man is walking down the street with a skateboard\n',
 'frame_2024-10-02_03-22-26_violence_0.00.png: a blue car is seen driving down the street\n',
 'frame_2024-10-02_03-22-27_violence_0.00.png: a blue car is seen driving down the street\n',
 'frame_2024-10-02_03-22-28_violence_0.00.png: a blue car is seen in the street on the camera\n',
 'frame_2024-10-02_03-22-29_violence_0.00.png: a blue car is seen in the street with a camera on it\n',
 'frame_2024-10-02_03-22-30_violence_0.00.png: a blue car is seen driving down the street\n',
 'frame_2024-10-02_03-22-31_violence_0.00.png: a blue car is seen in the street with a man on the sidewalk\n',
 'frame_2024-10-02_03-22-32_violence_0.00.png: a blue car is parked in the street next to a building\n',
 'frame_2024-10-02_03-22-33_violence_0.00.png: a car is seen in the middle of the road\n',
 'frame_2024-10-02_03-22-34_violence_0.00.png: a man is walking down the street in front of a blue car\n',
 'frame_2024-10-02_03-22-35_violence_32.95.png: a man is seen in the street with a blue car\n',
 'frame_2024-10-02_03-22-35_violence_43.78.png: a man is walking down the street with a blue car\n',
 'frame_2024-10-02_03-22-36_violence_0.00.png: a blue car is parked in the street next to a building\n',
 'frame_2024-10-02_03-22-37_violence_0.00.png: a car is parked in the street with a camera on it\n',
 'frame_2024-10-02_03-22-38_violence_0.00.png: a car is parked in the street and a camera is pointed at it\n',
 'frame_2024-10-02_03-22-39_violence_0.00.png: a car is parked in the street with a blue car\n',
 'frame_2024-10-02_03-22-40_violence_0.00.png: a car is parked in the street and a car is parked in the driveway\n',
 'frame_2024-10-02_03-22-41_violence_0.00.png: a car is parked in the street and a camera is pointed at it\n',
 'frame_2024-10-02_03-22-42_violence_0.00.png: a blue car is parked in the street next to a building\n',
 'frame_2024-10-02_03-22-43_violence_0.00.png: a man is standing in front of a car in the street\n',
 'frame_2024-10-02_03-22-44_violence_0.00.png: a man is standing next to a blue car\n',
 'frame_2024-10-02_03-22-45_violence_48.11.png: a man is standing next to a blue car\n',
 'frame_2024-10-02_03-22-45_violence_63.27.png: a man is being held by a police officer in the street\n',
 'frame_2024-10-02_03-22-46_violence_49.93.png: a man is being held by a police officer in the street\n',
 'frame_2024-10-02_03-22-47_violence_72.41.png: a man is being pulled by a man on a bike\n',
 'frame_2024-10-02_03-22-47_violence_80.33.png: a man is being pulled by a woman in a car\n',
 'frame_2024-10-02_03-22-48_violence_32.13.png: a woman is being chased by a man in a dress\n',
 'frame_2024-10-02_03-22-48_violence_81.10.png: a man is being held by a police officer in the street\n',
 'frame_2024-10-02_03-22-49_violence_0.00.png: a man is being chased by two men in the street\n',
 'frame_2024-10-02_03-22-50_violence_57.20.png: a man is seen in the street with a woman and a car\n',
 'frame_2024-10-02_03-22-51_violence_42.53.png: a man is standing next to a blue car\n',
 'frame_2024-10-02_03-22-51_violence_70.78.png: a man is walking down the street with a blue car\n',
 'frame_2024-10-02_03-22-52_violence_49.22.png: a blue car is seen in the street with a man on the ground\n',
 'frame_2024-10-02_03-22-53_violence_0.00.png: a man is seen in the back of a blue car\n',
 'frame_2024-10-02_03-22-54_violence_0.00.png: a man is being held by a police officer in the street\n',
 'frame_2024-10-02_03-22-55_violence_78.35.png: a man is being attacked by another man in the street\n',
 'frame_2024-10-02_03-22-55_violence_78.69.png: two men are fighting in the street with a woman\n',
 'frame_2024-10-02_03-22-57_violence_56.46.png: two men are fighting in the street with one man in the middle\n',
 'frame_2024-10-02_03-22-57_violence_67.31.png: a man is being attacked by another man in the street\n',
 'frame_2024-10-02_03-22-58_violence_65.20.png: two men are fighting in the street with one man holding a knife\n',
 'frame_2024-10-02_03-22-58_violence_89.86.png: two men are fighting in the street with one man in the middle\n',
 'frame_2024-10-02_03-22-59_violence_75.92.png: two men are fighting in the street with one man holding a baseball bat\n',
 'frame_2024-10-02_03-22-59_violence_91.59.png: two men are fighting in the street with one man holding a baseball bat\n',
 'frame_2024-10-02_03-23-00_violence_39.04.png: two men are fighting in the street with one holding a baseball bat\n',
 'frame_2024-10-02_03-23-01_violence_35.35.png: two men are walking in the street with a third man\n',
 'frame_2024-10-02_03-23-02_violence_0.00.png: two men are standing in the middle of the street\n',
 'frame_2024-10-02_03-23-02_violence_84.03.png: two men are fighting in the street with one man holding a baseball bat\n',
 'frame_2024-10-02_03-23-03_violence_0.00.png: a man is being attacked by a group of people\n',
 'frame_2024-10-02_03-23-03_violence_76.83.png: a man is being attacked by another man on the street\n',
 'frame_2024-10-02_03-23-04_violence_0.00.png: a man is walking down the street with a dog\n',
 'frame_2024-10-02_03-23-05_violence_44.13.png: a man is walking down the street with a group of people\n',
 'frame_2024-10-02_03-23-05_violence_47.80.png: a man is standing on the street with two other people\n',
 'frame_2024-10-02_03-23-06_violence_66.09.png: a man is standing on the street with a skateboard\n',
 'frame_2024-10-02_03-23-07_violence_0.00.png: a man is walking down the street with a red shirt on\n',
 'frame_2024-10-02_03-23-08_violence_0.00.png: a man is walking down the street with a group of people\n',
 'frame_2024-10-02_03-23-08_violence_28.97.png: a man is walking down the street with a woman and a child\n',
 'frame_2024-10-02_03-23-09_violence_0.00.png: a man is being held by a man and woman\n',
 'frame_2024-10-02_03-23-09_violence_40.16.png: a man is being led away by police officers\n',
 'frame_2024-10-02_03-23-10_violence_0.00.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-11_violence_0.00.png: a man is being held by a man in an orange shirt\n',
 'frame_2024-10-02_03-23-12_violence_0.00.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-12_violence_25.09.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-13_violence_46.89.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-13_violence_47.15.png: a man is being held by two men in a parking lot\n',
 'frame_2024-10-02_03-23-14_violence_0.00.png: a man is being held by police officers in the street\n',
 'frame_2024-10-02_03-23-15_violence_0.00.png: a man is being held by a man in a black shirt\n',
 'frame_2024-10-02_03-23-16_violence_0.00.png: a man is being held by police officers in the street\n',
 'frame_2024-10-02_03-23-17_violence_0.00.png: a man is being held by police officers in a parking lot\n',
 'frame_2024-10-02_03-23-18_violence_30.03.png: a man is being held by two men in an orange shirt\n',
 'frame_2024-10-02_03-23-18_violence_39.35.png: two men are seen in the street with a car\n',
 'frame_2024-10-02_03-23-19_violence_0.00.png: a man is walking down the street with a dog\n',
 'frame_2024-10-02_03-23-20_violence_43.10.png: two men are seen in the street with a dog\n',
 'frame_2024-10-02_03-23-20_violence_59.01.png: two men are seen in a parking lot with a car\n',
 'frame_2024-10-02_03-23-22_violence_0.00.png: two men are standing in the street with a car\n',
 'frame_2024-10-02_03-23-22_violence_66.90.png: two men are standing in the street with a car\n',
 'frame_2024-10-02_03-23-23_violence_0.00.png: two people are walking down the street with a car\n',
 'frame_2024-10-02_03-23-24_violence_29.70.png: two people are walking down the street with a car\n',
 'frame_2024-10-02_03-23-25_violence_0.00.png: two men are standing next to a car in the street\n',
 'frame_2024-10-02_03-23-25_violence_38.88.png: a man is standing next to a car with a woman standing next to him\n',
 'frame_2024-10-02_03-23-26_violence_0.00.png: a man is standing next to a car with a camera on it\n',
 'frame_2024-10-02_03-23-27_violence_0.00.png: a woman is seen standing next to a car in the street\n',
 'frame_2024-10-02_03-23-28_violence_30.19.png: a man is being pulled out of a car by a man\n',
 'frame_2024-10-02_03-23-29_violence_30.17.png: a man is being pulled out of a car by a woman\n',
 'frame_2024-10-02_03-23-30_violence_0.00.png: a man is walking down the street with a camera\n',
 'frame_2024-10-02_03-23-30_violence_35.31.png: a man is seen walking down the street with a skateboard\n',
 'frame_2024-10-02_03-23-31_violence_0.00.png: a man is being pulled by a car on a skateboard\n',
 'frame_2024-10-02_03-23-31_violence_31.05.png: a man is standing next to a car with a man standing next to him\n',
 'frame_2024-10-02_03-23-32_violence_0.00.png: a man is being held by a woman in the street\n',
 'frame_2024-10-02_03-23-33_violence_0.00.png: a man is being pulled away from a car by a woman\n',
 'frame_2024-10-02_03-23-33_violence_29.24.png: a man is being chased by a man in a red shirt\n',
 'frame_2024-10-02_03-23-34_violence_0.00.png: a man is being held by a police officer in the street\n',
 'frame_2024-10-02_03-23-35_violence_0.00.png: a man is being held by police officers in a parking lot\n',
 'frame_2024-10-02_03-23-36_violence_0.00.png: a man is standing next to a car with a man standing next to it\n',
 'frame_2024-10-02_03-23-36_violence_38.64.png: a man is being held by two men in a parking lot\n',
 'frame_2024-10-02_03-23-37_violence_0.00.png: a group of people standing around a car\n',
 'frame_2024-10-02_03-23-38_violence_0.00.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-39_violence_0.00.png: a man in a white dress is walking down the street\n',
 'frame_2024-10-02_03-23-40_violence_0.00.png: a man is walking down the street with a woman and a child\n',
 'frame_2024-10-02_03-23-41_violence_0.00.png: a man in a white robe is standing next to a car\n',
 'frame_2024-10-02_03-23-42_violence_0.00.png: a man is standing next to a car with a woman standing next to him\n',
 'frame_2024-10-02_03-23-43_violence_0.00.png: a man is standing next to a car with a woman standing next to him\n',
 'frame_2024-10-02_03-23-44_violence_55.93.png: a man is being held by two police officers\n',
 'frame_2024-10-02_03-23-44_violence_76.32.png: two men are standing next to a car in the street\n',
 'frame_2024-10-02_03-23-45_violence_52.32.png: a man is being held by two people in a parking lot\n',
 'frame_2024-10-02_03-23-46_violence_36.73.png: a man is being held by two people in the street\n',
 'frame_2024-10-02_03-23-47_violence_28.50.png: a man is being held by a woman and a man is standing next to a car\n',
 'frame_2024-10-02_03-23-47_violence_31.88.png: a man is standing next to a car with a woman standing next to him\n',
 'frame_2024-10-02_03-23-48_violence_0.00.png: a man is being attacked by a group of people\n',
 'frame_2024-10-02_03-23-48_violence_74.98.png: a man is being held by a woman in the street\n',
 'frame_2024-10-02_03-23-49_violence_77.49.png: a man is being held by two police officers in the street\n',
 'frame_2024-10-02_03-23-50_violence_38.98.png: a man is being held by a woman and a man is standing next to a car\n',
 'frame_2024-10-02_03-23-50_violence_47.39.png: a man is standing next to a car with a woman and two men\n',
 'frame_2024-10-02_03-23-51_violence_34.04.png: a man is being held by two people in a parking lot\n',
 'frame_2024-10-02_03-23-52_violence_0.00.png: a man is standing next to a car with a woman standing next to it\n',
 'frame_2024-10-02_03-23-52_violence_33.84.png: a man is being held by two men in the street\n',
 'frame_2024-10-02_03-23-53_violence_0.00.png: a man is walking down the street with a woman\n',
 'frame_2024-10-02_03-23-53_violence_37.01.png: a man is walking down the street with a dog\n',
 'frame_2024-10-02_03-23-54_violence_0.00.png: two men are walking down the street in front of a car\n',
 'frame_2024-10-02_03-23-55_violence_37.47.png: two men are walking down the street in front of a car\n',
 'frame_2024-10-02_03-23-56_violence_0.00.png: a man is walking down the street in front of a car\n',
 'frame_2024-10-02_03-23-56_violence_51.01.png: a man is walking down the street with a skateboard\n',
 'frame_2024-10-02_03-23-57_violence_0.00.png: a man is walking down the street with a blue car\n',
 'frame_2024-10-02_03-23-58_violence_0.00.png: the word we is blurred in blue light\n',
 'frame_2024-10-02_03-23-59_violence_0.00.png: we pay sign on a wall\n',
 'frame_2024-10-02_03-24-00_violence_0.00.png: a close up of the word cash on a wall\n',
 'frame_2024-10-02_03-24-01_violence_0.00.png: the word efr is shown on a white wall\n',
 'frame_2024-10-02_03-24-02_violence_0.00.png: we pay cash for videos\n',
 'frame_2024-10-02_03-24-03_violence_0.00.png: dailymail.co.uk - pay cash for videos\n',
 'frame_2024-10-02_03-24-04_violence_0.00.png: dailymail com - pay cash for videos\n',
 'frame_2024-10-02_03-24-05_violence_0.00.png: dailymail com - pay cash for videos\n',
 'frame_2024-10-02_03-24-06_violence_0.00.png: dailymail com - pay cash for videos\n',
 'frame_2024-10-02_03-24-07_violence_0.00.png: a crescent moon is shown in the dark sky\n',
 'frame_2024-10-02_03-24-08_violence_0.00.png: a picture of a plane flying in the dark with the words "m" and "m',
 'frame_2024-10-02_03-24-08_violence_40.41.png: a man walking in the rain with a car\n',
 'frame_2024-10-02_03-24-09_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-24-10_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-24-11_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-24-12_violence_0.00.png: a car is driving down the street in the city\n',
 'frame_2024-10-02_03-24-13_violence_0.00.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-14_violence_0.00.png: a man walking in the rain holding an umbrella\n',
 'frame_2024-10-02_03-24-15_violence_0.00.png: a person walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-16_violence_34.81.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-16_violence_59.83.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-17_violence_62.79.png: a couple of people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-24-18_violence_76.37.png: a man holding an umbrella walks down a street in the rain\n',
 'frame_2024-10-02_03-24-19_violence_0.00.png: a group of people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-24-20_violence_42.78.png: a group of people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-24-20_violence_76.49.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-21_violence_0.00.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-21_violence_41.78.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-22_violence_44.04.png: a man walking down the street with a car parked in the middle\n',
 'frame_2024-10-02_03-24-23_violence_49.24.png: a man walking down the street with a car parked in the middle\n',
 'frame_2024-10-02_03-24-24_violence_0.00.png: two men walking down the street in the rain\n',
 'frame_2024-10-02_03-24-25_violence_31.42.png: two people walking on the street in the rain\n',
 'frame_2024-10-02_03-24-25_violence_33.46.png: two people walking on the street in the rain\n',
 'frame_2024-10-02_03-24-26_violence_0.00.png: two people walking across the street in the rain\n',
 'frame_2024-10-02_03-24-27_violence_0.00.png: two people walking on the street in the rain\n',
 'frame_2024-10-02_03-24-28_violence_0.00.png: a car is driving down the street in the rain\n',
 'frame_2024-10-02_03-24-28_violence_46.88.png: a car is parked on the side of the road\n',
 'frame_2024-10-02_03-24-29_violence_48.61.png: a man walking down the street with a cane\n',
 'frame_2024-10-02_03-24-29_violence_48.75.png: a man walking down the street with a cane\n',
 'frame_2024-10-02_03-24-30_violence_36.51.png: a man is walking down the street in the rain\n',
 'frame_2024-10-02_03-24-30_violence_59.68.png: a man walking down the street with a cane\n',
 'frame_2024-10-02_03-24-31_violence_25.49.png: a man walking down the street with a cane\n',
 'frame_2024-10-02_03-24-32_violence_44.40.png: a man walking down the street in the rain\n',
 'frame_2024-10-02_03-24-33_violence_40.02.png: a man walking down the street with a cane\n',
 'frame_2024-10-02_03-24-33_violence_68.55.png: a man walking on the street in the rain\n',
 'frame_2024-10-02_03-24-34_violence_0.00.png: a man is walking down the street in the rain\n',
 'frame_2024-10-02_03-24-34_violence_29.85.png: a man is walking down the street in the rain\n',
 'frame_2024-10-02_03-24-35_violence_0.00.png: a street with cars parked on both sides of the road\n',
 'frame_2024-10-02_03-24-36_violence_25.15.png: a street with cars parked on both sides of the road\n',
 'frame_2024-10-02_03-24-36_violence_40.92.png: a car is driving down a wet street\n',
 'frame_2024-10-02_03-24-37_violence_0.00.png: a man is standing on the side of the road with a umbrella\n',
 'frame_2024-10-02_03-24-38_violence_66.77.png: a person walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-38_violence_75.37.png: a person walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-39_violence_58.92.png: a man and a woman walking in the rain\n',
 'frame_2024-10-02_03-24-39_violence_79.31.png: two people walking in the rain\n',
 'frame_2024-10-02_03-24-40_violence_0.00.png: a man and a woman walking in the rain\n',
 'frame_2024-10-02_03-24-41_violence_76.53.png: a car is driving down a street in the rain\n',
 'frame_2024-10-02_03-24-42_violence_72.48.png: a car is driving down the street\n',
 'frame_2024-10-02_03-24-42_violence_80.99.png: a car is driving down the street in the rain\n',
 'frame_2024-10-02_03-24-43_violence_0.00.png: a car is parked on the side of the road\n',
 'frame_2024-10-02_03-24-43_violence_32.70.png: a car is driving down a wet street\n',
 'frame_2024-10-02_03-24-45_violence_46.30.png: a car is driving down a wet street\n',
 'frame_2024-10-02_03-24-45_violence_60.51.png: a car is driving down the street in the rain\n',
 'frame_2024-10-02_03-24-46_violence_26.38.png: a man walking across the street with an umbrella\n',
 'frame_2024-10-02_03-24-47_violence_29.20.png: two men are walking down the street in the rain\n',
 'frame_2024-10-02_03-24-48_violence_0.00.png: a man walking across the street with an umbrella\n',
 'frame_2024-10-02_03-24-48_violence_30.28.png: a man walking in the rain with a car in the background\n',
 'frame_2024-10-02_03-24-49_violence_32.59.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-49_violence_63.09.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-50_violence_65.78.png: people walking in the rain\n',
 'frame_2024-10-02_03-24-51_violence_30.34.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-52_violence_73.55.png: a man and woman walking in the rain\n',
 'frame_2024-10-02_03-24-52_violence_73.68.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-53_violence_56.33.png: a man walking in the rain with a child\n',
 'frame_2024-10-02_03-24-54_violence_43.16.png: a car is parked on the side of the road\n',
 'frame_2024-10-02_03-24-55_violence_35.08.png: a man holding an umbrella in the rain\n',
 'frame_2024-10-02_03-24-55_violence_42.96.png: a man holding an umbrella in the rain\n',
 'frame_2024-10-02_03-24-56_violence_26.76.png: a woman walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-24-56_violence_40.32.png: a man is walking down the street with an umbrella\n',
 'frame_2024-10-02_03-24-57_violence_45.20.png: a car is parked on the side of the road\n',
 'frame_2024-10-02_03-24-58_violence_0.00.png: a car is parked on the side of the road\n',
 'frame_2024-10-02_03-24-59_violence_45.56.png: a couple of people walking down the street with umbrellas\n',
 'frame_2024-10-02_03-24-59_violence_75.71.png: a couple of people standing in the rain holding umbrellas\n',
 'frame_2024-10-02_03-25-00_violence_49.23.png: two people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-25-01_violence_80.04.png: a blurry picture of two people walking in the rain\n',
 'frame_2024-10-02_03-25-02_violence_35.14.png: three men walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-25-02_violence_74.54.png: three men walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-25-03_violence_49.26.png: people walking in the rain\n',
 'frame_2024-10-02_03-25-04_violence_0.00.png: a group of people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-25-05_violence_31.60.png: people walking in the rain\n',
 'frame_2024-10-02_03-25-05_violence_62.99.png: people walking in the rain\n',
 'frame_2024-10-02_03-25-06_violence_78.44.png: a group of people walking in the rain with umbrellas\n',
 'frame_2024-10-02_03-25-07_violence_83.38.png: people walking in the rain\n',
 'frame_2024-10-02_03-25-08_violence_53.03.png: a man walking in the rain with an umbrella\n',
 'frame_2024-10-02_03-25-08_violence_82.04.png: people walking in the rain\n',
 'frame_2024-10-02_03-25-09_violence_0.00.png: a man is walking down the street with an umbrella\n',
 'frame_2024-10-02_03-25-09_violence_66.16.png: a man walking down the street with an umbrella\n',
 'frame_2024-10-02_03-25-10_violence_46.10.png: a woman holding an umbrella in the rain\n',
 'frame_2024-10-02_03-25-11_violence_56.67.png: a woman holding an umbrella in the rain\n',
 'frame_2024-10-02_03-25-12_violence_0.00.png: a street with cars and people walking on it\n',
 'frame_2024-10-02_03-25-13_violence_37.75.png: a man is standing in the rain with an umbrella\n',
 'frame_2024-10-02_03-25-13_violence_54.65.png: a person holding an umbrella in the rain\n',
 'frame_2024-10-02_03-25-14_violence_77.36.png: a man and a woman walking down a street with an umbrella\n',
 'frame_2024-10-02_03-25-15_violence_29.44.png: a man holding an umbrella and walking down the street\n',
 'frame_2024-10-02_03-25-16_violence_0.00.png: a stick in the dark sky\n',
 'frame_2024-10-02_03-25-17_violence_0.00.png: the cctv people logo on a black and white sign\n',
 'frame_2024-10-02_03-25-18_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-19_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-20_violence_0.00.png: the cctv people logo on a black screen\n',
 'frame_2024-10-02_03-25-21_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-22_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-23_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-24_violence_0.00.png: the cctv people logo on a black background\n',
 'frame_2024-10-02_03-25-25_violence_0.00.png: the cctv people logo on a black screen\n',
 'frame_2024-10-02_03-25-26_violence_0.00.png: a tv screen showing a video of a man in a store\n',
 'frame_2024-10-02_03-25-27_violence_0.00.png: a tv screen showing a video of a man in a suit\n',
 'frame_2024-10-02_03-25-28_violence_0.00.png: a tv screen showing a video of a man in a suit\n',
 'frame_2024-10-02_03-25-29_violence_0.00.png: a tv screen showing a video of a man in a suit\n',
 'frame_2024-10-02_03-25-30_violence_0.00.png: a tv screen showing a video of a man in a suit\n',
 'frame_2024-10-02_03-25-31_violence_0.00.png: a black background with a white object in the middle\n',
 'frame_2024-10-02_03-25-33_violence_0.00.png: a black background with a white clock on it\n',
 'frame_2024-10-02_03-25-34_violence_0.00.png: a black background with a white square in the middle\n',
 'frame_2024-10-02_03-25-35_violence_0.00.png: a black background with a white square in the middle\n',
 'frame_2024-10-02_03-25-36_violence_0.00.png: a black background with a white object in the middle\n']


def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w', encoding='utf-8') as file:
        for item in data_list:
            file.write(f"{item}\n")  # Write each item on a new line

save_list_to_txt('/kaggle/working/captions.txt', captions)


In [8]:
# Function to continuously process new queries
def process_queries():
    last_processed_queries = set()  # Keep track of processed queries

    while True:
        # Step 2: Retrieve all queries from Firestore
        queries_ref = db.collection('queries')  # Adjust the collection name if necessary
        queries = queries_ref.stream()

        new_queries = []
        for query_doc in queries:
            query_text = query_doc.to_dict().get('query', '')
            if query_text and query_doc.id not in last_processed_queries:
                new_queries.append((query_doc.id, query_text))

        print(f"Found {len(new_queries)} new queries.")

        # Process new queries
        for query_id, query_text in new_queries:
            print(f"Processing query: {query_text}")

            # Find best matches for the query
            best_matches = find_best_matches(query_text, captions)

            # Extract only the file names from the best matches
            # Assuming best_matches contains strings like "frame_2024-10-01_19-37-12_violence_0.00.png: description"
            file_names = [match.split(':')[0] for match in best_matches]

            # Save response back to Firestore as an array
            db.collection('queries').document(query_id).set({
                'response': file_names,  # Save as a list of file names
                'processed': True,  # Mark as processed
                'timestamp': firestore.SERVER_TIMESTAMP  # Optional: add a server timestamp
            }, merge=True)

            print(f"Processed query {query_id}: Matches: {file_names}")

            # Add this query to the processed set
            last_processed_queries.add(query_id)

        # If there are no new queries, wait before checking again
        if not new_queries:
            print("No new queries found. Waiting for more...")
            time.sleep(5)  # Wait for a while before checking again

# Example usage
process_queries()


Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for more...
Found 0 new queries.
No new queries found. Waiting for 

NameError: name 'find_best_matches' is not defined